## Synthetic Data Generation - SWIFT MT103

Synthetic data for SWIFT MT103(Single Customer Credit Transfer) creates curated synthetic data which helps banks and other financial institutions to test their systems, do simulation exercises, train employees and do compliance testing without getting exposed to actual SWIFT messages. These synthetic swift messages mimic actual transactions in their format and structure. The synthetic data is generated for both the Mandatory and Optional tags, hence users can get data for every kind of transaction through message type MT103. The solution also gives users to get information of how a swift transaction would look like between certain countries, currencies and banks of their choice. Added to it users can enter custom banks for synthetic data generation. This would help generate data for specifically for some banks/financial institutions/countries.  

This sample notebook shows you how to execute Synthetic Data Generation - SWIFT MT103 Algorithm from AWS Marketplace 

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Synthetic Data Generation - SWIFT MT103. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Execute optimization model](#3.-Execute-optimization-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Execute model](#3.2-Execute-model)
    1. [Visualize Output](#3.3-Inspect-the-Output-in-S3)
1. [Clean-up](#4.-Clean-up)
	1. [Unsubscribe to the listing (optional)](#Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page **Synthetic Data Generation - SWIFT MT103**
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [2]:
algo_arn = "swift-listing-copy-11-21-copy-11-21"

### 2. Prepare dataset

In [2]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint
from PIL import Image

import numpy as np
import pandas as pd

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described for best results:
* Input File name should be input_zip.zip
* The input data files must contain one json file(containing countries,banks and currencies across which they wish to see transactions) and/or a csv file(optional) containing extra banks/currencies/countries which user wants to input.
* For detailed instructions, please refer sample notebook and algorithm input details.

#### B. Configure dataset

In [4]:
training_dataset="Input/input_zip.zip"

#### C. Upload datasets to Amazon S3

In [5]:
role = get_execution_role()

In [ ]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
# training input location
common_prefix = "swift-listing"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

## 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm. 

### 3.1 Set up environment

In [8]:
output_location = 's3://{}/swift-listing/{}'.format(bucket, 'Output')

### 3.2 Execute model

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [9]:
training_instance_type="ml.m5.4xlarge"

In [10]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="swift-listing-algo",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

2023-11-23 11:43:26 Starting - Starting the training job...
2023-11-23 11:43:41 Starting - Preparing the instances for trainingProfilerReport-1700739806: InProgress
......
2023-11-23 11:44:44 Downloading - Downloading input data...
2023-11-23 11:45:24 Training - Downloading the training image.........
2023-11-23 11:46:44 Training - Training image download completed. Training in progress.../usr/local/lib/python3.8/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
#015  0%|          | 0/100 [00:00<?, ?it/s]#015Sampling rows:   0%|     

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [ ]:
#output is available on following path
estimator.output_path

## Note: Inferencing is done within training pipeline. Real time inference endpoint/batch transform job is not required.

### 3.3 Inspect the Output in S3

In [12]:
parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')
response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)

In [13]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/Output/'+ estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

In [ ]:
s3_conn = boto3.client("s3")
bucket_name=bucket
with open('output.tar.gz', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder, f)
    print("Output file loaded from bucket")

In [15]:
with tarfile.open('output.tar.gz') as file:
    file.extractall('./output')

In [3]:
output_path = "output/output"
f = open(os.path.join(output_path,'synth_data.json'))
json_strings = json.load(f)  
# Since output are json strings, use this command to convert those into a dataframe

data = [json.loads(js) for js in json_strings]

df = pd.DataFrame(data)
df.head(10)


,ID,MESSAGE TYPE,SENDER,RECEIVER,:20:,:23B:,:23E:,:32A:,:33B:,:50A:,...,:56C:,:56D:,:57A:,:57B:,:57C:,:57D:,:59:,:59A:,:59F:,:71A:
0,0,fin.103,BCSIBRRSXXX,ABBYGB2LXXX,FXLEAPZPM01KVUK7,SPRI,PHOB/762293,"210615GBP48665201,10","BRL88337952,60",/N207261586664ABBYGB2LXXX,...,ABBYGB2LXXX,ABBYGB2LXXX SANTANDER UK PLC UNITED KINGDOM,BSCHBRSPXXX,BSCHBRSPXXX BRAZIL,BSCHBRSPXXX,BSCHBRSPXXX COOPERATIVA DE ECONOMIA E CREDITO ...,/U106978233790452\nRQPXWA IQWRMFLLQC\nStudio 3...,/C8662683602876880ABBYGB2LXXX,/Z15074572318\n1\PRIKNS RHZTIXQDXV\n2\67 Matth...,OUR
1,1,fin.103,ABBYGB2LXXX,ABBYGB2LXXX,ZVOYSU9TVDAJZFIE,SSTD,INTC,"210223GBP1906322,00","GBP58104217,20",/Q183511986306CEFBBRSP,...,CITIBRBRXXX,CITIBRBRXXX BANCO VOTORANTIM S.A. BRAZIL,BIBRBRSPXXX,BIBRBRSPXXX BRAZIL,BIBRBRSPXXX,BIBRBRSPXXX COOPERATIVA DE ECONOMIA E CREDITO ...,/Q464914633136643\nDYBDCF JFEENZYMHV\n9 Tony h...,/G9167462616123823BCSIBRRSXXX,/X69664235468\n1\SYVAAW ECTOFERYGE\n2\Studio 6...,OUR
2,2,fin.103,ABBYGB2LXXX,CEFBBRSP,QH5UIYRT3IBMVOAD,SPRI,REPA/605108496,"230719GBP66871040,80","BRL87865602,20",/R164580415745SAFRBRSPXXX,...,//IN64900886718,//IN22912286388BARBINBBXXX BANK OF INDIA INDIA,BNBRBRCFXXX,BNBRBRCFXXX BRAZIL,BNBRBRCFXXX,BNBRBRCFXXX BANIF - BCO INTERNACIONAL DO FUNCH...,/Y760468267396109\nMUZOEA CMRQFDBJPL\nFlat 41W...,/G1760042144048740BCSIBRRSXXX,/N90999937349\n1\UJYTVI RYIOJAGYSU\n2\Flat 3\n...,BEN
3,3,fin.103,ABBYGB2LXXX,BRBSBRDFXXX,M8OIUQDSJBE3QASK,SPRI,SDVA/617749034619,"210125GBP1927682,80","BRL833791,00",/X281917627097IBKLINBBXXX,...,ABBYGB2LXXX,ABBYGB2LXXX SANTANDER UK PLC UNITED KINGDOM,SCHABRSPXXX,SCHABRSPXXX BRAZIL,SCHABRSPXXX,SCHABRSPXXX BANCO ABC BRASIL S.A. BRAZIL,/K647852827514379\nDVGQNK RAIBSFPPYA\n324 Hugh...,/Z8069295087762904BCSIBRRSXXX,/R71746356574\n1\LYXCGW XWVPPCILHY\n2\86 Smith...,BEN
4,4,fin.103,ABBYGB2LXXX,TOPZBRRSXXX,AOBWCD1UYTWB9BGQ,SSTD,TELB/210698524760,"211031GBP1021,90","GBP4169903,20",/P149727015213BCSIBRRSXXX,...,RBOSGB2L,RBOSGB2L SANTANDER UK PLC UNITED KINGDOM,BCSIBRRSXXX,BCSIBRRSXXX BRAZIL,BCSIBRRSXXX,BCSIBRRSXXX COOPERATIVA DE CREDITO DE LIVRE AD...,/R983640271545602\nNEMPAL WYNNUXONJZ\n756 Shan...,/Z7382203151059893ABBYGB2LXXX,/Z79145593540\n1\FRKJSO RMOAOUKJQC\n2\40 Coope...,SHA
5,5,fin.103,ABBYGB2LXXX,FIBIBRSPXXX,ANWXLIBYFLZQXXZV,SPAY,CORT/447368421936443,"210815BRL64420187,10","GBP1000,70",/L790303295706BCSIBRRSXXX,...,ABBYGB2LXXX,ABBYGB2LXXX SANTANDER UK PLC UNITED KINGDOM,ABBYGB2LXXX,ABBYGB2LXXX UNITED KINGDOM,ABBYGB2LXXX,ABBYGB2LXXX SANTANDER UK PLC UNITED KINGDOM,/K637191281696508\nLHVXOL LARYYRANCX\nStudio 3...,/D7669059785920457ABBYGB2LXXX,/P84003842198\n1\UMPVOB JVEGBTGNZM\n2\33 Abbie...,BEN
6,6,fin.103,ABBYGB2LXXX,BCSIBRRSXXX,CTFTN1DQS1QFDMMU,SSTD,INTC/33879869576487,"210113GBP12551886,20","INR57951163,80",/D183619742388BSCHBRSPXXX,...,BCSIBRRSXXX,BCSIBRRSXXX BANCO MODAL S.A. BRAZIL,HSBCGB2L,HSBCGB2L United Kingdom,HSBCGB2L,HSBCGB2L SANTANDER UK PLC UNITED KINGDOM,/Q798842369185539\nYGEPTD XRRWNXCGDD\n096 Webb...,/H9619560889532704PRBABRSPXXX,/B82273443354\n1\CHCYXS ERBBRFSNSD\n2\426 Step...,OUR
7,7,fin.103,ABBYGB2LXXX,BMAXBRRJXXX,HI21KYAYNXQC7URS,CRTS,TELE,"220123GBP713562921,80","BRL450059556,50",/Q851345456772ABBYGB2LXXX,...,BSCHBRSPXXX,BSCHBRSPXXX BANCO INDUSTRIAL DO BRASIL S.A. BR...,ABBYGB2LXXX,ABBYGB2LXXX UNITED KINGDOM,ABBYGB2LXXX,ABBYGB2LXXX SANTANDER UK PLC UNITED KINGDOM,/T403180286817625\nYQELVS RPKTXGIUKH\n8 Wood r...,/J8851551463955614RBOSGB2L,/C38999741885\n1\KYVSOI UGQAXULANX\n2\Flat 0\n...,OUR
8,8,fin.103,CHASBRSPXXX,BCSIBRRSXXX,QVQNSGWZXD9X3SRH,SPRI,TELB,"231014GBP625289347,20","GBP6802034,00",/Y902149034188BCSIBRRSXXX,...,BCSIBRRSXXX,BCSIBRRSXXX COOP DE CREDITO MUTUO DOS COMERCIA...,ABBYGB2LXXX,ABBYGB2LXXX UNITED KINGDOM,ABBYGB2LXXX,ABBYGB2LXXX SANTANDER UK PLC UNITED KINGDOM,/R293491010222431\nWSMFVK UWBHSZIWIL\nStudio 4...,/A6271021174013027BCSIBRRSXXX,/F49686361103\n1\QYYBXT KYDCVSJHRU\n2\Flat 6\n...,BEN
9,9,fin.103,BCSIBRRSXXX,ABBYGB2LXXX,MK2RTRKITDVL3H30,CRTS,PHOB/2633386433,"210408G

### 4. Clean-up

#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.